def load_data(self, data_dir, data_type="train", reverse=False):
    with open("%s%s.txt" % (data_dir, data_type), "r") as f:
        data = f.read().strip().split("\n")
        data = [i.split('\t') for i in data]
        if reverse:
            data += [[i[2], i[1]+"_reverse", i[0]] for i in data]
    return data

In [ ]:
! ls

In [ ]:
! ls dataset/FB15K-237-N

In [ ]:
! sh TuckER/run.sh

In [ ]:
valid_relation_set = set()

In [ ]:
rel_list = ['/people/person/nationality', '/location/location/contains', '/people/person/place_lived', 
            '/people/deceased_person/place_of_death', '/people/person/ethnicity', '/people/ethnicity/people',
            '/business/person/company', '/people/person/religion', '/location/neighborhood/neighborhood_of',
            '/business/company/founders', '/people/person/children', '/location/administrative_division/country',
            '/location/country/administrative_divisions', '/business/company/place_founded', '/location/us_county/county_seat']

for i in range(len(rel_list)):
    valid_relation_set.add(rel_list[i])


In [ ]:
valid_relation_set

In [ ]:
'people/person/religion' not in valid_relation_set

In [ ]:
import os
train_data = open("./TuckER/data/nyt10/nyt10_train.txt", "r")

In [ ]:
lines = train_data.readlines()

In [ ]:
len(lines)

In [ ]:
lines[522610]

In [ ]:
def convert_data(line):

    relation_left = '''"relation": "'''
    relation_right = '''", "h":'''
    relation_left_idx = line.find(relation_left)
    relation_right_idx = line.find(relation_right)

    head_left = '''"h": {"id":'''
    head_right = '''", "name":'''
    head_name_right = '''", "pos":'''
    head_left_idx = line.find(head_left) 
    head_right_idx = line.find(head_right)
    head_name_right_idx = line.find(head_name_right)

    tail_left = '''"t": {"id":'''
    tail_right = '''", "name":'''
    tail_name_right = '''", "pos":'''
    tail_left_idx = line.find(tail_left)
    tail_right_idx = line.find(tail_right, tail_left_idx)
    tail_name_right_idx = line.find(tail_name_right, tail_right_idx)
    
    
    head = line[head_left_idx+13:head_right_idx]
    head_name = line[head_right_idx+12:head_name_right_idx]
    relation = line[relation_left_idx+13:relation_right_idx]
    tail = line[tail_left_idx+13:tail_right_idx]
    tail_name = line[tail_right_idx+12:tail_name_right_idx]
    
    head = "/" + head.replace(".", "/")
    tail = "/" + tail.replace(".", "/")
    
    return head, relation, tail, head_name, tail_name


In [ ]:
convert_data(lines[13434])

In [ ]:
data = ""
test_data = ""
val_data = ""
for i in range(len(lines)):
    head, relation, tail, head_name, tail_name = convert_data(lines[i])
#     print(relation)
    if relation not in valid_relation_set:
        continue
    tmp = head + '\t' + relation + '\t' + tail + '\n'
    if i % 13 == 0:
        test_data += tmp
    elif i % 13 == 1:
        val_data += tmp
    else:
        data += tmp

In [ ]:
len(lines) * 0.85

In [ ]:
txt_file = open("./TuckER/data/nyt10/train.txt", 'w', encoding='utf-8')
print(data, file=txt_file)

In [ ]:
train_data_ = open("./TuckER/data/nyt10/nyt10_train.txt", "r")
test_data_ = open("./TuckER/data/nyt10/nyt10_test.txt", "r")

train_lines = train_data_.readlines()
test_lines = test_data_.readlines()

In [ ]:
from tqdm import tqdm
def entity_to_label(entity2label, lines):
    for i in tqdm(range(len(lines))):
        head, relation, tail, head_name, tail_name = convert_data(lines[i])
        if relation not in valid_relation_set:
            continue
        tmp1 = head + '\t' + head_name + '\n'
        tmp2 = tail + '\t' + tail_name + '\n'
        if tmp1 not in entity2label:
            entity2label.append(tmp1)
        if tmp2 not in entity2label:
            entity2label.append(tmp2)

In [ ]:
test_lines

In [ ]:
# make entity2label
entity2label = []

entity_to_label(entity2label, train_lines)
entity_to_label(entity2label, test_lines)

entity2label

In [ ]:
for i in range(len(test_lines)):
    head, relation, tail, head_name, tail_name = convert_data(test_lines[i])
    if relation not in valid_relation_set:
        continue
    tmp1 = head + '\t' + head_name + '\n'
    tmp2 = tail + '\t' + tail_name + '\n'
    if tmp1 not in entity2label:
        entity2label.append(tmp1)
    if tmp2 not in entity2label:
        entity2label.append(tmp2)

In [ ]:
e2l_data = ""
for i in range(len(entity2label)):
    e2l_data += entity2label[i]


e2l_file = open("./dataset/nyt10/entity2label.txt", 'w', encoding='utf-8')
print(e2l_data, file=e2l_file)

In [ ]:
for i in range(len(test_lines)):
    head, relation, tail, head_name, tail_name = convert_data(test_lines[i])
    if relation not in valid_relation_set:
        continue
    tmp = head + '\t' + relation + '\t' + tail + '\n'
    if i % 2 == 0:
        test_data += tmp
    else:
        val_data += tmp

In [ ]:
test_txt_file = open("./TuckER/data/nyt10/test.txt", 'w', encoding='utf-8')
print(test_data, file=test_txt_file)
val_txt_file = open("./TuckER/data/nyt10/valid.txt", 'w', encoding='utf-8')
print(val_data, file=val_txt_file)

In [ ]:
test_data_ = open("./dataset/nyt10/test.txt", "r")
valid_data_ = open("./dataset/nyt10/valid.txt", "r")

test_lines = test_data_.readlines()
valid_lines = valid_data_.readlines()

In [ ]:
test_lines[0].split('\t')

In [ ]:
# make test/valid negative data
import random
head_rel = []
tails = []
test_neg = []
for i in range(len(test_lines)):

    tmp1 = test_lines[i].split('\t')[0] + '\t' + test_lines[i].split('\t')[1] + '\t'
    tmp2 = test_lines[i].split('\t')[2]
    head_rel.append(tmp1)
    tails.append(tmp2)
    
for i in range(len(head_rel)):
    random_tail = random.choice(tails)
    tmp = head_rel[i] + random_tail
    tails.remove(random_tail)
    test_neg.append(tmp)

test_neg_data = ""
for i in range(len(test_neg)):
    test_neg_data += test_neg[i]
test_neg_file = open("./dataset/nyt10/test_neg.txt", 'w', encoding='utf-8')
print(test_neg_data, file=test_neg_file)

In [ ]:
# make test/valid negative data
import random
head_rel = []
tails = []
valid_neg = []
for i in range(len(valid_lines)):

    tmp1 = valid_lines[i].split('\t')[0] + '\t' + valid_lines[i].split('\t')[1] + '\t'
    tmp2 = valid_lines[i].split('\t')[2]
    head_rel.append(tmp1)
    tails.append(tmp2)
    
for i in range(len(head_rel)):
    random_tail = random.choice(tails)
    tmp = head_rel[i] + random_tail
    tails.remove(random_tail)
    valid_neg.append(tmp)

valid_neg_data = ""
for i in range(len(valid_neg)):
    valid_neg_data += valid_neg[i]
valid_neg_file = open("./dataset/nyt10/valid_neg.txt", 'w', encoding='utf-8')
print(valid_neg_data, file=valid_neg_file)

In [ ]:
test_neg_data = ""
valid_neg_data = ""
for i in range(len(test_neg)):
    test_neg_data += test_neg[i]
for i in range(len(valid_neg)):
    valid_neg_data += valid_neg[i]

test_neg_file = open("./dataset/nyt10/test_neg.txt", 'w', encoding='utf-8')
print(test_neg_data, file=test_neg_file)

valid_neg_file = open("./dataset/nyt10/valid_neg.txt", 'w', encoding='utf-8')
print(valid_neg_data, file=valid_neg_file)

In [ ]:
import torch
torch.cuda.device_count()

In [ ]:
import torch
import sys
print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)
print('__CUDA VERSION')
from subprocess import call
# call(["nvcc", "--version"]) does not work
! nvcc --version
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__Devices')
call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
print('Active CUDA Device: GPU', torch.cuda.current_device())

print ('Available devices ', torch.cuda.device_count())
print ('Current cuda device ', torch.cuda.current_device())

In [ ]:
! sh nyt_10.sh

In [ ]:
!nvidia-smi